In [49]:
!pip install pandas scikit-learn nltk tensorflow
!pip install spacy
!python -m spacy download en_core_web_sm

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/61/5a/6506a36428a07f9abdebd8b18a3c815a097a201586bd37fafbfc0677f571/spacy-3.8.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/be/a2/34f2508058b6ae9c213736a36e5a7dbdc3e5b92e305b83d025a56e70f899/murmurhash-1.0.11-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/p

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
import nltk
import pickle
import spacy
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/ary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
dataset = "Internship Tracker - ML_Dataset.csv"
# Load the dataset
data = pd.read_csv(dataset)

# Check the first few rows of your dataset
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   email subject                   64 non-null     object
 1   email body                      64 non-null     object
 2   company name (expected output)  64 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


,email subject,email body,company name (expected output)
count,64,64,64
unique,36,37,27
top,Thank you for your application to HHAeXchange!,"Hi Aaryaman, Thank you for your interest in HH...",Boostability
freq,3,3,5


In [48]:
# Check for missing values
data.isnull().sum()

# Drop rows with missing values (if necessary)
data.dropna(subset=['email subject', 'email body', 'company name (expected output)'], inplace=True)

# Optionally, inspect cleaned data
data.head()


,email subject,email body,company name (expected output)
0,Jane Street Application Received,"Hello, Thank you for applying for a position a...",Jane Street
1,Thanks for applying to Stripe!,"Hi Aaryaman, Thanks so much for submitting you...",Stripe
2,"Aaryaman, Thanks for Applying for Salesforce's...","Hi Aaryaman, Hooray! You have officially appli...",Salesforce
3,"Aaryaman, your application was sent to America...",Your application was sent to American Institut...,American Institute of Insurance Research
4,Your application was viewed by [P1] Games,Your application was viewed by [P1] Games ͏ ͏ ...,[P1] Games


In [58]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Define a function to clean the text
def clean_text(text):
    # Tokenize and lower case the text
    text = text.lower()
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return " ".join(cleaned_tokens)

# Apply the cleaning function to both 'email subject' and 'email body'
for i in range(len(data)):
    print(clean_text(data['email body'][i]))
data['cleaned_subject'] = data['email subject'].apply(clean_text)
data['cleaned_body'] = data['email body'].apply(clean_text)


hello thank applying position jane street email confirm received resume touch shortly regarding status thank interest jane street
hi aaryaman thanks much submitting application software engineer intern role reminder recorded location preference cohort preference applicable winter summer based application responses one application required high volume applications university recruiting team review applications rolling basis may cause delay response times appreciate understanding meantime encourage check quick guide stripe culture 0 blog 1 linkedin page 2 get feel 're stripe hope connect soon stripe 0 https //stripe.com/jobs/life-at-stripe 1 https //stripe.com/blog 2 https //www.linkedin.com/company/stripe please note inbox accept replies warm responses received
hi aaryaman hooray officially applied summer 2025 intern software engineer opening salesforce thank letting us know ’ interested discovering new opportunities us good news – information cloud member recruiting team personally rev

In [64]:
X = data[['cleaned_subject', 'cleaned_body']]  # Features
y = data['company name (expected output)']    # Target variable

y.value_counts()

# Combine subject and body for better predictions
X.loc[:, 'combined_text'] = X['cleaned_subject'] + " " + X['cleaned_body']
X['combined_text'].value_counts()

/var/folders/kq/cs03j9553ng9l_hc6_yh2qyw0000gn/T/ipykernel_36482/3355244113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'combined_text'] = X['cleaned_subject'] + " " + X['cleaned_body']


combined_text
thank application hhaexchange hi aaryaman thank interest hhaexchange received application software engineer intern position delighted would like learn opportunity hiring team currently reviewing applications qualifications match needs role receive email one recruiters schedule phone interview thank taking time apply role hhaexchange best regards talent acquisition team                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [9]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Combine subject and body into one single column of text
X_train_combined = X_train['cleaned_subject'] + " " + X_train['cleaned_body']
X_test_combined = X_test['cleaned_subject'] + " " + X_test['cleaned_body']

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the training data, transform the test data
X_train_tfidf = vectorizer.fit_transform(X_train_combined)

X_test_tfidf = vectorizer.transform(X_test_combined)

X_train_tfidf.shape, X_test_tfidf.shape

((51, 646), (13, 646))

In [52]:
# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
nlp_model = spacy.load("en_core_web_sm")

# Train the model
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [53]:
def extract_organization(text):
    """
    Extract organization names from text using spaCy's NER.
    """
    doc = nlp_model(text)
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return orgs if orgs else ["Unknown"]  # Return "Unknown" if no organization found


In [67]:
X.loc[:, 'predicted company'] = X['combined_text'].apply(extract_organization)

X['predicted company'].value_counts()

predicted company
[Unknown]                                                                                                                                                                                                                                                                                  16
[united states, united states, united states, georgia institute technology]                                                                                                                                                                                                                11
[ibm, united states, georgia institute technology]                                                                                                                                                                                                                                          4
[united states, united states, georgia institute technology]                                                                

In [68]:
# Display the predictions
for i, row in X.iterrows():
    print(f"Email Subject: {row['cleaned_subject']}")
    print(f"Email Body: {row['cleaned_body']}")
    print(f"Predicted Company Name: {row['predicted_company']}")
    print("------")

Email Subject: jane street application received
Email Body: hello thank applying position jane street email confirm received resume touch shortly regarding status thank interest jane street
Predicted Company Name: ['jane street application']
------
Email Subject: thanks applying stripe
Email Body: hi aaryaman thanks much submitting application software engineer intern role reminder recorded location preference cohort preference applicable winter summer based application responses one application required high volume applications university recruiting team review applications rolling basis may cause delay response times appreciate understanding meantime encourage check quick guide stripe culture 0 blog 1 linkedin page 2 get feel 're stripe hope connect soon stripe 0 https //stripe.com/jobs/life-at-stripe 1 https //stripe.com/blog 2 https //www.linkedin.com/company/stripe please note inbox accept replies warm responses received
Predicted Company Name: ['Unknown']
------
Email Subject: aa

In [15]:
# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7692307692307693
Classification Report:
                  precision    recall  f1-score   support

        Blooket       1.00      1.00      1.00         1
   Boostability       1.00      1.00      1.00         1
       Coinbase       0.00      0.00      0.00         0
      Collabera       1.00      1.00      1.00         1
        Cranium       1.00      1.00      1.00         2
         Fortra       1.00      1.00      1.00         1
   HHAeXchange!       1.00      1.00      1.00         1
      Hangtight       0.00      0.00      0.00         0
    Jane Street       1.00      1.00      1.00         1
    Mission Bit       0.00      0.00      0.00         1
Rayca Precision       1.00      1.00      1.00         1
     TuneTunnel       0.00      0.00      0.00         2
     [P1] Games       1.00      1.00      1.00         1

       accuracy                           0.77        13
      macro avg       0.69      0.69      0.69        13
   weighted avg       0.77      0

/Users/ary/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ary/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ary/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ary/anaconda3/lib/python3.11/site-p

In [17]:
# Save the trained model using pickle
with open('company_name_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the vectorizer as well
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)


In [18]:
# Load the model and vectorizer from disk
with open('company_name_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

with open('vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

In [26]:
def predict_company_name(email_subject, email_body):
    # Combine subject and body
    email_content = email_subject + ' ' + email_body
    
    # Transform the email content using the loaded vectorizer
    email_tfidf = loaded_vectorizer.transform([email_content])
    
    # Predict the company name
    prediction = loaded_model.predict(email_tfidf)
    
    return prediction[0]


In [44]:
# Test with an example email
email_subject = "Applied to amazon"
email_body = "Hello, we would like to schedule a meeting with your team to discuss an interview with Amazon. Please let us know your availability. Thank you."

company_name = predict_company_name(email_subject, email_body)
print(f"Predicted Company Name: {company_name[0]}")

Predicted Company Name: Coinbase
